# 2-Layer Neural Network | Text Colour Predictor

Task:
- Feed-forward neural network.
- Feed RGB values of 'background colour'.
- Predict if dark or light coloured text should be used over the RGB colour to make the text readable.

## Import libraries:

In [ ]:
# Import libraries:
import numpy as np # For linear algebra.
import pandas as pd # For data processing, CSV file I/O (e.g. pd.read_csv).
import matplotlib.pyplot as plt # For data visualisation.
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
%matplotlib inline


## RGB class and tool functions:

In [ ]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass


class InputError(Error):
    """Exception raised for errors in the input.

    Attributes:
        expr -- Input expression in which the error occurred.
        msg  -- Explanation of the error."""
    def __init__(self, expr, msg):
        self.expr = expr
        self.msg = msg


class RGB():
    """Defined with values for RGB as input.
    
    Attributes:
        RGB -- Input RGB values should range from 0 to 255.
        hex -- Automatically converts RGB to hex values."""
    def __init__(self, R, G, B):
        for X in [R, G, B]:
            if (X < 0) or (X > 255):
                raise InputError(X, 'Not an RGB value.')
        self.R = R
        self.G = G
        self.B = B
        self.RGB = (R, G, B)
        self.hex = '#{:02X}{:02X}{:02X}'.format(self.R,self.G,self.B)


def generate_RGB_data(X, extreme=False):
    """Generates a list filled with X number of RGB class values.
    Optional: generate cols that are v. dark + v. light for training.
    
    Attributes:
        X -- Number of desired RGB instances.
        extreme -- Boolean to generate v. dark + v. light cols."""
    if extreme == True:
        cols = []
        for x in range(X):
            minimum = 127*(x%2)
            maximum = 255-(127*(not x%2))
            rgb = RGB(np.random.randint(low=minimum, high=maximum),
                      np.random.randint(low=minimum, high=maximum),
                      np.random.randint(low=minimum, high=maximum))
            cols.append(rgb)
        return cols
                        
    else:
        return [RGB(np.random.randint(0, 255),
                    np.random.randint(0, 255),
                    np.random.randint(0, 255))
                for i in range(X)]


def display_RGB_colour(colour, font_col='#000'):
    """Will draw a box of given colour;
    and fill with text of given font colour.
    
    Attributes:
        colour -- String containing a RGB or hex value.
        font_col -- String containing a RGB or hex value."""
    img = Image.new(mode='RGB', size=(100, 100), color=colour)
    img_draw = ImageDraw.Draw(img)
    img_draw.text((36, 45), 'Text', fill=font_col)
    plt.imshow(img)
    plt.show();


## Data visualisation:

In [ ]:
# Test the RGB class and data visualisation tool functions:
colours = generate_RGB_data(X=2, extreme=True)

for colour in colours:
    print('RGB:', colour.RGB, 'Hex:', colour.hex)
    display_RGB_colour(colour=colour.RGB, font_col='#fff')


## Generate data:

In [ ]:
np.random.seed(2) # Optional: set seed for data generation.

data = pd.DataFrame([x.RGB for x in generate_RGB_data(X=200, extreme=1)],
                     columns=['R', 'G', 'B'])

display('Training set:', data)


## Assign Labels (NB: lazy method)

In [ ]:
# Import libraries:
from sklearn.cluster import AgglomerativeClustering


In [ ]:
clusterer = AgglomerativeClustering(n_clusters=2, linkage='ward').fit(data.values)
y = clusterer.labels_

print(y)


In [ ]:
for i, label in enumerate(y[:2]):
    
    if label == 0: # NB: must check for most appriate label-to-class assignment.
        print('---> light text')
        display_RGB_colour(colour=tuple(data.iloc[i, :]), font_col='#fff')
        
    else:
        print('---> dark text')
        display_RGB_colour(colour=tuple(data.iloc[i, :]), font_col='#000')


## Train/test split:

In [ ]:
# Import libraries:
from sklearn.model_selection import train_test_split


In [ ]:
# Split data into training & testing sets:
train, test = train_test_split(data.join(pd.Series(y, name='y')))

display(train.head())
display(test.head())


## Building the NN:

In [ ]:
# Define Neuron class:
class NeuralNetwork():
    def __init__(self, X, y, bias=1, eta=0.1, w1=None, w2=None, num_nodes=2, num_hidden=2, linear=False):
        self.X = X
        self.y = y
        self.eta = eta
        self.w1 = np.random.rand(num_nodes, self.X.shape[0]) if w1 is None else w1
        self.w2 = np.random.rand(self.y.shape[0], num_nodes) if w2 is None else w2
        self.b1 = np.ones((self.w1.shape[0], self.X.shape[1]), dtype=float)*bias
        self.b2 = np.ones(self.y.shape, dtype=float)*bias
        self.output = np.zeros(self.y.shape)
        self.linear = linear
    
    def activ_func(self, x):
        if self.linear is True: # For linear:
            return x
        else: # For sigmoid:
            return 1.0/(1.0 + np.exp(-x))
    
    def forwardpass(self):
        self.layer1 = self.activ_func(np.dot(self.w1, self.X) + self.b1)
        self.output = self.activ_func(np.dot(self.w2, self.layer1) + self.b2)
        
    def activ_deriv(self, x):
        if self.linear is True: # For linear:
            return 1
        else: # For sigmoid:
            return self.activ_func(x)*(1-self.activ_func(x))
    
    def error_deriv(self):
        return -(self.y-self.output)
    
    def backprop(self):
        """Apply chain rule to find derivative of loss function."""
        # Output layer:
        big_delta = self.error_deriv() * self.activ_deriv(x=np.dot(self.w2, self.layer1))
        output_unit = -self.eta * np.dot(big_delta, self.layer1.T)

        # Hidden layer:
        sml_delta = np.dot(big_delta.T, self.w2).T * self.activ_deriv(x=np.dot(self.w1, self.X))        
        hidden_unit = -self.eta * np.dot(sml_delta, self.X.T)
        
        """Update the weights and biases with the derivative (slope) of the loss function."""
        # Weights:
        self.w2 += output_unit
        self.w1 += hidden_unit
        
        # Biases:
        self.b2 += -self.eta * big_delta * self.b2
        self.b1 += -self.eta * sml_delta * self.b1
    
    def fit(self, Xs, ys, iterations=20):
        y_preds = []
        for i, X in enumerate(Xs): # Per data point:
            self.X = X
            self.y = ys[i]
            
            for i in range(iterations): # Per iteration:
                self.forwardpass()
                self.backprop()
                
            y_preds.append(self.output)
            
        return np.array(y_preds)
    
    def predict(self, Xs):
        y_preds = []
        for X in Xs:
            self.X = X
            self.forwardpass()
            
            y_preds.append(self.output)
            
        return np.array(y_preds)
    
    def display_test_results(self, Xs, y_preds):
        for i, y in enumerate(y_preds):
            if y == 0:
                print(y, '---> light text')
                display_RGB_colour(colour=tuple(Xs[i, :]), font_col='#fff')

            else:
                print(y, '---> dark text')
                display_RGB_colour(colour=tuple(Xs[i, :]), font_col='#000')


## Use the NN on dummy example:

In [ ]:
# Setup inputs:
X = np.array([0, 1]).reshape((2,1))
y = np.array([1, 0]).reshape((2,1))
w1 = np.array([[-1, 0],
               [0, 1]], dtype=float)
w2 = np.array([[1, 0],
               [-1, 1]], dtype=float)

# Initialise NN:
NN = NeuralNetwork(X=X,
                   y=y,
                   bias=1,
                   eta=0.1,
                   w1=w1,
                   w2=w2,
                   num_nodes=2,
                   linear=True)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.w1, NN.w2))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))


## Train the NN on training set:

In [ ]:
# Setup inputs:
input_X = train.values[0, :3].reshape((3,1))
input_y = train.values[0, 3].reshape((1,1))
print('X:\n{}\ny:\n{}'.format(input_X, input_y))

# Initialise NN:
NN = NeuralNetwork(X=input_X,
                   y=input_y,
                   bias=1,
                   eta=0.75,
                   w1=None,
                   w2=None,
                   num_nodes=10,
                   linear=False)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.w1, NN.w2))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.output))


In [ ]:
# Setup inputs:
tr_i = train.shape[0]
train_Xs = train.values[:, :3].reshape((tr_i, 3, 1))
train_ys = train.values[:, 3].reshape((tr_i, 1))
print('Shapes of inputs:', train_Xs.shape, train_ys.shape)

# Train NN:
train_y_preds = NN.fit(Xs=train_Xs, ys=train_ys, iterations=20)
print('Shape of y_preds:', train_y_preds.shape)


In [ ]:
train_y_preds

In [ ]:
# Training results:
train_results = pd.DataFrame({'y_true': train.y.values,
                              'y_pred': np.round(train_y_preds).reshape((tr_i,)).astype(int),
                              'same': train.y.values == np.round(train_y_preds).reshape((tr_i,)).astype(int)})

display(train_results.head())

print('{}% error'.format(round(len(train_results[train_results.same==False]) / len(train_results) * 100)))


In [ ]:
# Setup inputs:
te_i = test.shape[0]
test_Xs = test.values[:, :3].reshape((te_i, 3, 1))
test_ys = test.values[:, 3].reshape((te_i, 1))
print('Shapes of inputs:', test_Xs.shape, test_ys.shape)

# Test NN:
test_y_preds = NN.predict(Xs=test_Xs)
print('Shape of y_preds:', test_y_preds.shape)


In [ ]:
# Testing results:
test_results = pd.DataFrame({'y_true': test.y.values,
                             'y_pred': np.round(test_y_preds).reshape((te_i,)).astype(int),
                             'same': test.y.values == np.round(test_y_preds).reshape((te_i,)).astype(int)})

display(test_results.head(15))

print('{}% error'.format(round(len(test_results[test_results.same==False]) / len(test_results) * 100)))


In [ ]:
# Display results:
NN.display_test_results(Xs=test.values[:, :3],
                        y_preds=test_results.y_pred.values)


# NN via scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier


In [ ]:
sklearn_NN = MLPClassifier(activation='logistic')

sklearn_NN


In [ ]:
sklearn_NN.fit(X=train.iloc[:, :3].values, y=train.y.values)


In [ ]:
sklearn_y_preds = sklearn_NN.predict(X=test.iloc[:, :3].values)


In [ ]:
sklearn_results = pd.DataFrame({'y_true': test.y.values,
                                'y_pred': sklearn_y_preds,
                                'same': test.y.values == sklearn_y_preds})

display(sklearn_results.head())

print('{}% error'.format(round(len(sklearn_results[sklearn_results.same==False]) / len(sklearn_results) * 100)))


In [ ]:
# The end.